In [6]:
import numpy as np
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams, cm
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import matplotlib.animation as animation
from IPython.display import HTML

In [7]:
nx = 41
ny = 41
L  =  1
H =  1

dx  =  L/(nx-1)
dy=  H/(ny-1)

l2_norm = 1
l1_target = 1e-8

In [8]:
#l=ind de la position en l
l=int(L/dx)   #dx=dy

def border_cond(l, delta_x):
    #neumann cond
    psi[1:-1,1] = 1/4*psi[1:-1,2]     #en x = 0 (forward 2nd order)
    psi[1:-1,l-1] = 1/4*psi[1:-1,l-2]   #en x = l
    psi[1,1:-1]= 1/4*psi[2,1:-1]         #en y=0
    psi[l-1,1:-1]=1/2*(1/2*psi[l-2,i]-delta_x)     #en y = L

In [9]:
def conjugate_gradient_biharm(p, l, delta_x, dx, dy, l2_target):
    '''Performs cg relaxation
    Assumes Dirichlet boundary conditions p=0
    
    Parameters:
    ----------
    p : 2D array of floats
        Initial guess
    b : 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction
    l2_target: float
        exit criterion
        
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    '''
    ny, nx = p.shape
    r  = numpy.zeros((ny,nx)) # residual
    Ad  = numpy.zeros((ny,nx)) # to store result of matrix multiplication 
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Step-0 We compute the initial residual and 
    # the first search direction is just this residual
    
    r[1:-1,1:-1]= 1/20 * (8*(p[1:-1,2:] + p[2:,1:-1] + p[1:-1,:-2]+ \
                        p[:-2, 1:-1])-2*(p[:-2,2:]+p[2:,2:]+\
                        p[:-2,:-2])-(p[1:-1,3:]+p[3:,1:-1]+\
                        p[1:-1,:-3]+p[:-3,1:-1]))
    
    
    #r[1:-1,1:-1] = 4*p[1:-1,1:-1] - \
        #p[1:-1,2:] - p[1:-1,:-2] - p[2:, 1:-1] - p[:-2, 1:-1]
    d = r.copy()
    rho = numpy.sum(r*r)
    
    Ad[1:-1,1:-1]= -1/20 * (8*(d[1:-1,2:] - d[2:,1:-1] - d[1:-1,:-2]- \
                        d[:-2, 1:-1])+2*(d[:-2,2:]-d[2:,2:]-\
                        d[:-2,:-2])+(d[1:-1,3:]-d[3:,1:-1]-\
                        d[1:-1,:-3]-d[:-3,1:-1]))
    
    #Ad[1:-1,1:-1] = -4*d[1:-1,1:-1]+d[1:-1,2:]+d[1:-1,:-2]+\
        #d[2:, 1:-1] + d[:-2, 1:-1]
    sigma = numpy.sum(d*Ad)
    
    # Iterations
    while l2_norm > l2_target:

        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        
        alpha = rho/sigma

        p = pk + alpha*dk
        r = rk- alpha*Ad
        
        rhop1 = numpy.sum(r*r)
        beta = rhop1 / rho
        rho = rhop1
        
        d = r + beta*dk
        Ad[1:-1,1:-1]= -1/20 * (8*(d[1:-1,2:] - d[2:,1:-1] - d[1:-1,:-2]- \
                        d[:-2, 1:-1])+2*(d[:-2,2:]-d[2:,2:]-\
                        d[:-2,:-2])+(d[1:-1,3:]-d[3:,1:-1]-\
                        d[1:-1,:-3]-d[:-3,1:-1]))
       # Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
         #   d[2:, 1:-1] + d[:-2, 1:-1]
        sigma = numpy.sum(d*Ad)
        
        # BCs are automatically enforced
        border_cond(l, delta_x)
        
        l2_norm = L2_rel_error(pk,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of CG iterations: {0:d}'.format(iterations))
    return p, l2_conv     
